In [1]:
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.ax import AxSearch
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import nevergrad as ng
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
#from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from zoopt import ValueType
import torch
import adabelief_pytorch
global_checkpoint_period=np.inf

In [2]:
def train_TREC(config):
    import torch
    from torchtext import data
    from torchtext import datasets
    import random

    SEED = 1234
    savedPath = os.getcwd()
    os.chdir('/home/antoine/Projet/NovelTuning')
    
    
    #torch.manual_seed(SEED)
    #torch.backends.cudnn.deterministic = True

    TEXT = data.Field(tokenize = 'spacy')
    LABEL = data.LabelField()

    train_data, test_data = datasets.TREC.splits(TEXT, LABEL,root='data/trec', fine_grained=False)

    train_data, valid_data = train_data.split(random_state = random.seed(SEED))

    MAX_VOCAB_SIZE = 25_000


    sigmoid_func_uniq = get_sigmoid_func(config.get("sigmoid_func", 0))

    
    TEXT.build_vocab(train_data, 
                     max_size = MAX_VOCAB_SIZE, 
                     vectors = 'glove.6B.100d', 
                     unk_init = torch.Tensor.normal_)

    LABEL.build_vocab(train_data)

    os.chdir(savedPath)
    
    BATCH_SIZE = 64

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
        (train_data, valid_data, test_data), 
        batch_size = BATCH_SIZE,
        device = device)



   
    import torch.nn as nn
    import torch.nn.functional as F

    class CNN(nn.Module):
        def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                     dropout, pad_idx):

            super().__init__()

            self.embedding = nn.Embedding(vocab_size, embedding_dim)

            self.convs = nn.ModuleList([
                                        nn.Conv2d(in_channels = 1, 
                                                  out_channels = n_filters, 
                                                  kernel_size = (fs, embedding_dim)) 
                                        for fs in filter_sizes
                                        ])

            self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

            self.dropout = nn.Dropout(dropout)

        def forward(self, text):

            #text = [sent len, batch size]

            text = text.permute(1, 0)

            #text = [batch size, sent len]

            embedded = self.embedding(text)

            #embedded = [batch size, sent len, emb dim]

            embedded = embedded.unsqueeze(1)

            #embedded = [batch size, 1, sent len, emb dim]

            conved = [sigmoid_func_uniq(conv(embedded)).squeeze(3) for conv in self.convs]

            #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]

            pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

            #pooled_n = [batch size, n_filters]

            cat = self.dropout(torch.cat(pooled, dim = 1))

            #cat = [batch size, n_filters * len(filter_sizes)]

            return self.fc(cat)
    INPUT_DIM = 7503
    EMBEDDING_DIM = 100
    N_FILTERS = 100
    FILTER_SIZES = [2,3,4]
    OUTPUT_DIM = len(LABEL.vocab)
    DROPOUT = 0.5
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

    model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

 #   print(f'The model has {count_parameters(model):,} trainable parameters')

    pretrained_embeddings = TEXT.vocab.vectors

    model.embedding.weight.data.copy_(pretrained_embeddings)

    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    import torch.optim as optim

    #optimizer = optim.Adam(model.parameters())
    if(optimizer_is_adam == True):
        optimizer = torch.optim.Adam(model.parameters(), lr=config.get("lr", 0.01), 
                                 betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                         eps=config.get("eps", 1e-08), weight_decay=config.get("weight_decay", 0), 
                                         amsgrad=True)
    else: 
        optimizer = adabelief_pytorch.AdaBelief(model.parameters(), lr=config.get("lr", 0.01), 
                                 betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                         eps=config.get("eps", 1e-08), weight_decay=config.get("weight_decay", 0))
    
    
    criterion = nn.CrossEntropyLoss()

    model = model.to(device)
    criterion = criterion.to(device)

    def categorical_accuracy(preds, y):
        """
        Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
        """
        max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
        correct = max_preds.squeeze(1).eq(y)
        return correct.sum() / torch.FloatTensor([y.shape[0]])


    def train(model, iterator, optimizer, criterion):

        epoch_loss = 0
        epoch_acc = 0

        model.train()

        for batch in iterator:

            optimizer.zero_grad()

            predictions = model(batch.text)

            loss = criterion(predictions, batch.label)

            acc = categorical_accuracy(predictions, batch.label)

            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        return epoch_loss / len(iterator), epoch_acc / len(iterator)

    def evaluate(model, iterator, criterion):

        epoch_loss = 0
        epoch_acc = 0

        model.eval()

        with torch.no_grad():

            for batch in iterator:

                predictions = model(batch.text)

                loss = criterion(predictions, batch.label)

                acc = categorical_accuracy(predictions, batch.label)

                epoch_loss += loss.item()
                epoch_acc += acc.item()

        return epoch_loss / len(iterator), epoch_acc / len(iterator)

    import time

    def epoch_time(start_time, end_time):
        elapsed_time = end_time - start_time
        elapsed_mins = int(elapsed_time / 60)
        elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
        return elapsed_mins, elapsed_secs


    best_valid_loss = float('inf')

    for e in range(ITERATIONS):

        start_time = time.time()

        train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)


        tune.report(loss=valid_loss)

        if e % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")

 #       print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  #      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
        





In [3]:
def categorical_accuracy(preds, y):
        """
        Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
        """
        max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
        correct = max_preds.squeeze(1).eq(y)
        return correct.sum() / torch.FloatTensor([y.shape[0]])
def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
class SentimentRNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_size, n_layers,
             drop_prob, sigmoid , vocab_size):
        super(SentimentRNN, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)
        #self.lstm = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)

        self.dropout = nn.Dropout(0.3)

        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        hidden= tuple([each.data for each in hidden])
        batch_size = x.size(0)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                        weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


class SentimentRNN1(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_size, n_layers,
             drop_prob, sigmoid , vocab_size):
        super(SentimentRNN1, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                        #    dropout=drop_prob, batch_first=True)
        self.lstm = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)

        self.dropout = nn.Dropout(0.3)

        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        hidden = hidden.data
        batch_size = x.size(0)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                        weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        (a,b) = hidden
        return a
    
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx,sigmoid):

        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)
        
        self.sigmoid = sigmoid
        self.sig = nn.Sigmoid()


    def forward(self, text):

        #text = [sent len, batch size]

       # text = text.permute(1, 0)
        #We want already have batch size, len for sentiment!!!!!
        #text = [batch size, sent len]

        embedded = self.embedding(text)

        #embedded = [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1)

        #embedded = [batch size, 1, sent len, emb dim]

        conved = [self.sigmoid(conv(embedded)).squeeze(3) for conv in self.convs]

        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        #pooled_n = [batch size, n_filters]

        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        sig_out = self.sig(self.fc(cat))

        # reshape to be batch_size first
        sig_out = sig_out.view(50, -1)
        #sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return np.squeeze(sig_out)

In [5]:
class SentimentRNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_size, n_layers,
             drop_prob, sigmoid , vocab_size):
        super(SentimentRNN, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                            dropout=drop_prob, batch_first=True)
        #self.lstm = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)

        self.dropout = nn.Dropout(0.3)

        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        hidden= tuple([each.data for each in hidden])
        batch_size = x.size(0)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)

        # sigmoid function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels

        # return last sigmoid output and hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                        weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


class SentimentRNN1(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_size, n_layers,
             drop_prob, sigmoid , vocab_size):
        super(SentimentRNN1, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        #self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers,
                        #    dropout=drop_prob, batch_first=True)
        self.lstm = nn.GRU(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)

        self.dropout = nn.Dropout(0.3)

        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        self.sigmoid = sigmoid

    def forward(self, x, hidden):
        x = x.permute(1, 0)
        hidden = hidden.data
        batch_size = x.size(0)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        # dropout and fully connected layer
        out = self.dropout(self.sigmoid(lstm_out))
        out = self.fc(out)
        # sigmoid function
        #sig_out = self.sigmoid(out)
        # reshape to be batch_size first
        
        #sig_out = sig_out.view(batch_size, -1)
        
        sig_out = out[-batch_size-1:-1:, :] # get last batch of labels
        #sig_out = sig_out.view(-1,self.output_size)
        #print(sig_out.shape)

        # return last sigmoid output and hidden state
        return sig_out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                        weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        (a,b) = hidden
        return a
    
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx,sigmoid):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = sigmoid
        self.sig = nn.Sigmoid()

    def forward(self, text):
        #text = [sent len, batch size]
        text = text.permute(1, 0) ######################################
        #We want already have batch size, len for sentiment!!!!!
        #text = [batch size, sent len]
        embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]
        conved = [self.sigmoid(conv(embedded)).squeeze(3) for conv in self.convs]
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)
        sig_out = self.sig(self.fc(cat))
        # reshape to be batch_size first
        sig_out = sig_out.view(50, -1)
        #sig_out = sig_out[:, -1] # get last batch of labels
        # return last sigmoid output and hidden state
        return np.squeeze(sig_out)

In [6]:
def train_TREC(config):
    import torch
    from torchtext import data
    from torchtext import datasets
    import random

    SEED = 1234
    savedPath = os.getcwd()
    os.chdir('/home/antoine/Projet/NovelTuning')
    
    
    #torch.manual_seed(SEED)
    #torch.backends.cudnn.deterministic = True

    TEXT = data.Field(tokenize = 'spacy')
    LABEL = data.LabelField()

    train_data, test_data = datasets.TREC.splits(TEXT, LABEL,root='data/trec', fine_grained=False)

    train_data, valid_data = train_data.split(random_state = random.seed(SEED))

        
    MAX_VOCAB_SIZE = 25_000


    sigmoid_func_uniq = get_sigmoid_func(config.get("sigmoid_func", 0))

    
    TEXT.build_vocab(train_data, 
                     max_size = MAX_VOCAB_SIZE, 
                     vectors = 'glove.6B.100d', 
                     unk_init = torch.Tensor.normal_)

    LABEL.build_vocab(train_data)

    os.chdir(savedPath)
    
    
    
    BATCH_SIZE = 64
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
        (train_data, valid_data, test_data), 
        batch_size = BATCH_SIZE,
        device = device)



    # First checking if GPU is available
    train_on_gpu=torch.cuda.is_available()


    sigmoid_func_uniq = get_sigmoid_func(config.get("sigmoid_func", 0))       
        
        
    INPUT_DIM = 7503
    embedding_dim = 100
    N_FILTERS = int(round(config.get("nfilter",100)))
    FILTER_SIZES = [int(round(config.get("filter_1",2))),
                    int(round(config.get("filter_2",3))),
                   int(round(config.get("filter_3",4)))]
    output_size = len(LABEL.vocab)
    DROPOUT = 0.5
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
    vocab_size=7503
    hidden_dim = int(round(config.get("hidden_dim",64)))
    n_layers =  0+ int( round(config.get("n_layer",1)))
    #model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX,sigmoid_func_uniq)
    modeln = 1

    if(modeln<1/3):
        model = SentimentRNN(embedding_dim, hidden_dim, output_size, n_layers,
                            config.get("droupout_prob",0.1),
                            sigmoid_func_uniq, vocab_size)   

    elif(modeln<2/3):
          model = SentimentRNN1(embedding_dim, hidden_dim, output_size, n_layers,
                            config.get("droupout_prob",0.1),
                            sigmoid_func_uniq, vocab_size)

    else:
        model = CNN(vocab_size, embedding_dim, hidden_dim, FILTER_SIZES, 
                output_size, config.get("droupout_prob",0.1), PAD_IDX,sigmoid_func_uniq)
    
    # loss and optimization functions 
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

 #   print(f'The model has {count_parameters(model):,} trainable parameters')
    
    pretrained_embeddings = TEXT.vocab.vectors
    
    model.embedding.weight.data.copy_(pretrained_embeddings)

    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

    model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)



    criterion = nn.CrossEntropyLoss()

    model = model.to(device)
    criterion = criterion.to(device)


    epochs = ITERATIONS # 3-4 is approx where I noticed the validation loss stop decreasing

    net = model
    
    if(config.get("adam",0)>0.5):
        optimizer = torch.optim.Adam(net.parameters(), lr=config.get("lr", 0.01), 
           weight_decay=config.get("weight_decay", 0), 
                                         amsgrad=True)
    else: 
        optimizer = adabelief_pytorch.AdaBelief(net.parameters(), lr=config.get("lr", 0.01), 
                              weight_decay=config.get("weight_decay", 0))
    

    clip=5 # gradient clipping


    def trainlstm():
         # initialize hidden state
        h = net.init_hidden(BATCH_SIZE)

        # batch loop
        for inputs, labels in train_loader:

            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            
            
            
            h = tuple([each.data for each in h])
            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), labels.float())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()

    def traingru():
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        net.train()
        #for (data, target) in train_loader:
        h = net.init_hidden(64)
        for batch in train_iterator:
            if(batch.text.size(1)!=64):
                return 
            


           # h = tuple([each.data for each in h])
            h = h.data
            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(batch.text, h)
     #       print(output.shape)
            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), batch.label)
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()
       
    
    def train_cnn():
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        net.train()
        #for (data, target) in train_loader:
        for batch in train_iterator:
            # We set this just for the example to run quickly.
           # if(train_on_gpu):
            #    batch.text = batchtext.cuda()

            optimizer.zero_grad()
            output = net(batch.text)
        #    print(output.shape)

            loss = criterion(output.squeeze(), batch.label)

            loss.backward()
            optimizer.step()
    
    def test_lstm():
       
        val_h = net.init_hidden(batch_size)
        val_losses = []
        net.eval()
        for inputs, labels in valid_loader:

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history

            val_h = tuple([each.data for each in val_h])
            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            output, val_h = net(inputs, val_h)
            val_loss = criterion(output.squeeze(), labels.float())

            val_losses.append(val_loss.item())

        net.train()
        return np.mean(val_losses) 

            
    def test_gru():
        
        val_losses = []
        net.eval()
        for batch in test_iterator:
            val_h = net.init_hidden(batch.text.size(1))

            val_h = val_h.data

            output, val_h = net(batch.text, val_h)
            acc = categorical_accuracy(output, batch.label)
            val_losses.append(acc.item())

        net.train()
    
        return np.mean(val_losses) 
    
    def test_cnn(test_iterator):
        val_losses = []
        net.eval()
        for batch in test_iterator:
         #   if(train_on_gpu):
        #        batch = batch.cuda()
            output = net(batch.text)
            acc = categorical_accuracy(output, batch.label)
            val_losses.append(acc.item())

         
        net.train()
    
        return np.mean(val_losses)     
            

    for e in range(ITERATIONS):

        train_cnn()
        acc=test_cnn(valid_iterator)
        test=test_cnn(test_iterator)
        tune.report(mean_accuracy=acc,loss=test)



In [7]:
#Configs
parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke_test", action="store_false", help="Finish not quickly for testing")
args, _ = parser.parse_known_args()          
        
    
    
experiment_metrics = dict(metric="mean_accuracy", mode="max")
#experiment_metrics = dict(metric="loss", mode="min")


ITERATIONS = 20 #20
NUM_TUNED=  128 #128
    

#i is in [0;1]
#We want all values between 0 and 1
def get_sigmoid_func(i):
    if(i<0.33):
        return nn.ReLU()
    elif(i<0.67):
        return nn.Tanh()
    else:
        return nn.Sigmoid()

    

tune_kwargs = {
    "num_samples": NUM_TUNED if args.smoke_test else NUM_TUNED,
    "config": {
    "steps": ITERATIONS,  
     "lr": tune.uniform(1e-8, 0.1 )
    #   ,  "embedding": tune.uniform(64, 1024),

      ,   "weight_decay": tune.uniform(1e-8, 0.1)
      ,  "sigmoid_func":tune.uniform(0,1),
        "hidden_dim":tune.uniform(32.,512.),
        "n_layer":tune.uniform(1,3),
        "droupout_prob":tune.uniform(0,0.5)  
     ,   "adam":tune.uniform(0,1)
     ,   "filter_1":tune.uniform(1,4)
     ,   "filter_2":tune.uniform(1,4)
    ,   "filter_3":tune.uniform(1,4)
     ,   "nfilter":tune.uniform(32,512)
   #   ,  "model":tune.uniform(0.34,0.66)
    }
}





In [8]:
for i in range(0,0):
    for j in range(0,1):
        x = train_TREC
        f_HyperOpt(x)
        f_BayesOpt(x)
        f_AX(x)
        f_NeverGrad(x)
        f_BOHB(x)
        f_Random(x)
        print("all worked with " + str(x)+  " !")
        
for i in range(0,2):  
        x = train_TREC_zo
        experiment_metrics = dict(metric="mean_accuracy", mode="min")

        f_ZOOpt(x)
    

NameError: name 'train_TREC_zo' is not defined

In [ ]:
def f_HyperOpt(dataset):
    
    scheduler = AsyncHyperBandScheduler(**experiment_metrics)
    algo = HyperOptSearch(**experiment_metrics)
    bayesopt= ConcurrencyLimiter(algo, max_concurrent=8)

    tune.run(dataset, **tune_kwargs , scheduler = scheduler,  name="hyper", search_alg=bayesopt)

In [ ]:
def f_BayesOpt(dataset):
    
    scheduler = AsyncHyperBandScheduler(**experiment_metrics)
    algo = BayesOptSearch(**experiment_metrics)
    bayesopt = ConcurrencyLimiter(algo, max_concurrent=8)

    tune.run(dataset, **tune_kwargs , scheduler = scheduler, name="bayes",  search_alg=bayesopt)

In [ ]:
def f_AX(dataset):
    
   
    if __name__ == "__main__":
                
        algo = AxSearch(
            max_concurrent=3, #was working with 2
            **experiment_metrics
        )
        scheduler = AsyncHyperBandScheduler(**experiment_metrics)
        algo = ConcurrencyLimiter(algo, max_concurrent=8)

        tune.run(
            dataset,       name="ax",
            search_alg=algo,
            scheduler=scheduler,
            **tune_kwargs)

        
#        algo = AxSearch(
#            **experiment_metrics
#        )

        
#        scheduler = AsyncHyperBandScheduler()
#        tune.run(
#            dataset,
#            **experiment_metrics,
#            search_alg=algo,
#            scheduler=scheduler,
#            **tune_kwargs)


In [ ]:

# TODO We are interested in multiple Population based algorithms from nevergrad, and certainly not in OnePlusOne. 
def f_NeverGrad(dataset):
    algo = NevergradSearch(
    optimizer=ng.optimizers.CMA
    # space=space,  # If you want to set the space manually
    )
    algo = ConcurrencyLimiter(algo, max_concurrent=8)

    scheduler = AsyncHyperBandScheduler()

    tune.run(
        dataset,
        **experiment_metrics,
      #  name="nevergrad",
        search_alg=algo, name="ng",
        scheduler=scheduler,
        **tune_kwargs) 
    

In [ ]:
def f_BOHB(dataset):

    bohb_hyperband = HyperBandForBOHB(
        time_attr="training_iteration",
        max_t=100,
        reduction_factor=2,
        **experiment_metrics)

    bohb_search = TuneBOHB(
        # space=config_space, 
        max_concurrent=8,
        **experiment_metrics)

    tune.run(
        dataset,
       # config=config, 
        scheduler=bohb_hyperband,name="bohb",
        search_alg=bohb_search,       
         **tune_kwargs)
        #num_samples=NUM_TUNED,
       # stop={"training_iteration": 100})
    
    
    

In [ ]:
def f_Random(dataset):
    
    algo = NevergradSearch(
    optimizer=ng.optimizers.RandomSearch,
    # space=space,  # If you want to set the space manually
    )
    algo = ConcurrencyLimiter(algo, max_concurrent=8)

    scheduler = AsyncHyperBandScheduler()

    tune.run(
        dataset,
        **experiment_metrics,
      #  name="nevergrad",
        search_alg=algo,   name="random",    
        scheduler=scheduler,
        **tune_kwargs) 
    

In [ ]:
def f_ZOOpt(dataset):

    dim_dict = {
        "lr": (ValueType.CONTINUOUS, [0, 1], 1e-2),
        "momentum": (ValueType.CONTINUOUS, [0,1, 0.9], 1e-2)
    }

    zoopt_search_config = {
        "parallel_num": 8,  # how many workers to parallel
    }

    

    zoopt_search = ZOOptSearch(
    algo="Asracos",  # only support Asracos currently
    #dim_dict=dim_dict,
    budget=ITERATIONS,
    #dim_dict=dim_dict,
   #     **zoopt_search_config,
    **experiment_metrics)
    zoopt_search = ConcurrencyLimiter(zoopt_search, max_concurrent=8)

    scheduler = AsyncHyperBandScheduler(**experiment_metrics)

   
    tune.run(dataset,
 #        config = config,
    search_alg=zoopt_search,
   # num_samples= ITERATIONS,
    scheduler=scheduler,       
    #         paralell_num=4,
    name="zoopt_search", 
              **tune_kwargs
    )

In [ ]:
def train_TREC_zo(config):
    import torch
    from torchtext import data
    from torchtext import datasets
    import random

    SEED = 1234
    savedPath = os.getcwd()
    os.chdir('/home/antoine/Projet/NovelTuning')
    
    
    #torch.manual_seed(SEED)
    #torch.backends.cudnn.deterministic = True

    TEXT = data.Field(tokenize = 'spacy')
    LABEL = data.LabelField()

    train_data, test_data = datasets.TREC.splits(TEXT, LABEL,root='data/trec', fine_grained=False)

    train_data, valid_data = train_data.split(random_state = random.seed(SEED))

        
    MAX_VOCAB_SIZE = 25_000


    sigmoid_func_uniq = get_sigmoid_func(config.get("sigmoid_func", 0))

    
    TEXT.build_vocab(train_data, 
                     max_size = MAX_VOCAB_SIZE, 
                     vectors = 'glove.6B.100d', 
                     unk_init = torch.Tensor.normal_)

    LABEL.build_vocab(train_data)

    os.chdir(savedPath)
    
    
    
    BATCH_SIZE = 64
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
        (train_data, valid_data, test_data), 
        batch_size = BATCH_SIZE,
        device = device)



    # First checking if GPU is available
    train_on_gpu=torch.cuda.is_available()


    sigmoid_func_uniq = get_sigmoid_func(config.get("sigmoid_func", 0))       
        
        
    INPUT_DIM = 7503
    embedding_dim = 100
    N_FILTERS = int(round(config.get("nfilter",100)))
    FILTER_SIZES = [int(round(config.get("filter_1",2))),
                    int(round(config.get("filter_2",3))),
                   int(round(config.get("filter_3",4)))]
    output_size = len(LABEL.vocab)
    DROPOUT = 0.5
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
    vocab_size=7503
    hidden_dim = int(round(config.get("hidden_dim",64)))
    n_layers =  0+ int( round(config.get("n_layer",1)))
    #model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX,sigmoid_func_uniq)
    modeln = 1

    if(modeln<1/3):
        model = SentimentRNN(embedding_dim, hidden_dim, output_size, n_layers,
                            config.get("droupout_prob",0.1),
                            sigmoid_func_uniq, vocab_size)   

    elif(modeln<2/3):
          model = SentimentRNN1(embedding_dim, hidden_dim, output_size, n_layers,
                            config.get("droupout_prob",0.1),
                            sigmoid_func_uniq, vocab_size)

    else:
        model = CNN(vocab_size, embedding_dim, hidden_dim, FILTER_SIZES, 
                output_size, config.get("droupout_prob",0.1), PAD_IDX,sigmoid_func_uniq)
    
    # loss and optimization functions 
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)

 #   print(f'The model has {count_parameters(model):,} trainable parameters')
    
    pretrained_embeddings = TEXT.vocab.vectors
    
    model.embedding.weight.data.copy_(pretrained_embeddings)

    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

    model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)



    criterion = nn.CrossEntropyLoss()

    model = model.to(device)
    criterion = criterion.to(device)


    epochs = ITERATIONS # 3-4 is approx where I noticed the validation loss stop decreasing

    net = model
    
    if(config.get("adam",0)>0.5):
        optimizer = torch.optim.Adam(net.parameters(), lr=config.get("lr", 0.01), 
           weight_decay=config.get("weight_decay", 0), 
                                         amsgrad=True)
    else: 
        optimizer = adabelief_pytorch.AdaBelief(net.parameters(), lr=config.get("lr", 0.01), 
                              weight_decay=config.get("weight_decay", 0))
    

    clip=5 # gradient clipping


    def trainlstm():
         # initialize hidden state
        h = net.init_hidden(BATCH_SIZE)

        # batch loop
        for inputs, labels in train_loader:

            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            
            
            
            h = tuple([each.data for each in h])
            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), labels.float())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()

    def traingru():
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        net.train()
        #for (data, target) in train_loader:
        h = net.init_hidden(64)
        for batch in train_iterator:
            if(batch.text.size(1)!=64):
                return 
            


           # h = tuple([each.data for each in h])
            h = h.data
            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(batch.text, h)
     #       print(output.shape)
            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), batch.label)
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()
       
    
    def train_cnn():
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        net.train()
        #for (data, target) in train_loader:
        for batch in train_iterator:
            # We set this just for the example to run quickly.
           # if(train_on_gpu):
            #    batch.text = batchtext.cuda()

            optimizer.zero_grad()
            output = net(batch.text)
        #    print(output.shape)

            loss = criterion(output.squeeze(), batch.label)

            loss.backward()
            optimizer.step()
    
    def test_lstm():
       
        val_h = net.init_hidden(batch_size)
        val_losses = []
        net.eval()
        for inputs, labels in valid_loader:

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history

            val_h = tuple([each.data for each in val_h])
            if(train_on_gpu):
                inputs, labels = inputs.cuda(), labels.cuda()

            output, val_h = net(inputs, val_h)
            val_loss = criterion(output.squeeze(), labels.float())

            val_losses.append(val_loss.item())

        net.train()
        return np.mean(val_losses) 

            
    def test_gru():
        
        val_losses = []
        net.eval()
        for batch in test_iterator:
            val_h = net.init_hidden(batch.text.size(1))

            val_h = val_h.data

            output, val_h = net(batch.text, val_h)
            acc = categorical_accuracy(output, batch.label)
            val_losses.append(acc.item())

        net.train()
    
        return np.mean(val_losses) 
    
    def test_cnn(test_iterator):
        val_losses = []
        net.eval()
        for batch in test_iterator:
         #   if(train_on_gpu):
        #        batch = batch.cuda()
            output = net(batch.text)
            acc = categorical_accuracy(output, batch.label)
            val_losses.append(acc.item())

         
        net.train()
    
        return np.mean(val_losses)     
            

    for e in range(ITERATIONS):

        train_cnn()
        acc=test_cnn(valid_iterator)
        test=test_cnn(test_iterator)
        tune.report(mean_accuracy=1-acc,loss=test)

